In [28]:
%matplotlib inline
import pandas as pd
import numpy as np
df_original = pd.read_csv('Cleaned_2012-18_playerBoxScore.csv')

In [2]:
pd.set_option('display.max_columns', 999)

In [3]:
df_original.describe(include='all')

,next_gmDays,pre_gmDays,gmTime,teamDayOff,playMin,playHeight,playWeight,playPTS,playAST,playTO,playSTL,playBLK,playPF,playFGA,playFGM,play2PA,play2PM,play3PA,play3PM,playFTA,playFTM,playORB,playDRB,playTRB,playFG%,play2P%,play3P%,playFT%,opptDayOff,gmYear,gmMonth,gmWeek,gmDay,gmDayofweek,gmDayofyear,gmIs_month_end,gmIs_month_start,gmIs_quarter_end,gmIs_quarter_start,gmIs_year_end,gmIs_year_start,gmElapsed,playBYear,playBMonth,playBWeek,playBDay,playBDayofweek,playBDayofyear,playBIs_month_end,playBIs_month_start,playBIs_quarter_end,playBIs_quarter_start,playBIs_year_end,playBIs_year_start,playBElapsed,IsStarter,teamConfIsEast,opptConfIsEast,teamRsltIsWin,opptRsltltIsWin,teamLocIsHome,opptLocIsHome,playPos_C,playPos_F,playPos_G,playPos_PF,playPos_PG,playPos_SF,playPos_SG,teamDiv_Atlantic,teamDiv_Central,teamDiv_Northwest,teamDiv_Pacific,teamDiv_Southeast,teamDiv_Southwest,opptDiv_Atlantic,opptDiv_Central,opptDiv_Northwest,opptDiv_Pacific,opptDiv_Southeast,opptDiv_Southwest,age,Seats,Year_Open,pre_gm_miles
count,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,1.540140e+05,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,1.540140e+05,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.0,154014.0,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000,154014.000000
mean,2.349851,2.390562,15.682535,2.076396,22.988676,79.011609,217.932941,9.855974,2.185892,1.297285,0.737595,0.483047,1.880232,7.989767,3.676510,5.714013,2.853195,2.275754,0.823315,2.209448,1.679627,1.011116,3.146272,4.157388,0.425244,0.444552,0.214111,0.432078,1.815926,2015.110068,5.939525,23.937525,15.288493,3.008726,165.169192,0.031984,0.037412,0.011324,0.010311,0.005792,0.003597,1.437743e+09,1987.966613,6.322906,25.714818,15.236563,2.904126,176.530588,0.018881,0.035179,0.006649,0.016193,0.000779,0.003974,5.821386e+08,0.475931,0.496254,0.499435,0.591686,0.408314,1.0,0.0,0.161362,0.001078,0.000649,0.214383,0.212461,0.192125,0.217941,0.166764,0.164154,0.166556,0.167141,0.165336,0.170049,0.166323,0.166478,0.166972,0.167829,0.166634,0.165764,26.615347,19000.412014,1998.628300,1561.301450
std,1.277117,1.296341,5.880749,1.049777,10.690116,3.431578,25.942307,7.932366,2.532465,1.382802,0.983238,0.883062,1.441477,5.502435,2.982805,4.556545,2.624615,2.538952,1.243163,2.810060,2.287218,1.408193,2.737350,3.552377,0.247963,0.295154,0.294300,0.429615,1.067600,1.743989,4.547303,19.885091,9.004586,1.951334,139.444160,0.175958,0.189770,0.105809,0.101017,0.075883,0.059868,5.352373e+07,4.562999,3.289666,14.367896,8.765910,1.991734,100.624052,0.136107,0.184232,0.081269,0.126219,0.027902,0.062912,1.440824e+08,0.499422,0.499988,0.500001,0.491523,0.491523,0.0,0.0,0.367866,0.032813,0.025473,0.410395,0.409051,0.393972,0.412849,0.372766,0.370416,0.372580,0.373103,0.371484,0.375677,0.372371,0.372511,0.372952,0.373715,0.372650,0.371870,5.073903,1021.014056,11.162248,1783.445556
min,0.000000,0.000000,1.000000,0.000000,0.000000,69.000000,149.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0

In [4]:
new_col_list =  df_original.loc[:,'playPTS':'playTRB'].columns

In [5]:
%%time
for col in new_col_list:
    new_col = col +'_permin'
    df_original[new_col] =df_original[col] / df_original['playMin']

CPU times: user 263 ms, sys: 276 ms, total: 539 ms
Wall time: 186 ms


In [6]:
df_original.drop(columns=new_col_list, inplace=True)
df = df_original.copy()

In [7]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)

In [8]:
df.fillna(0, inplace=True)

In [51]:
#df.to_csv('permin.csv')

In [9]:
X = np.array(df.drop('play3P%', axis=1))
y = np.array(df['play3P%'])

In [10]:
X.shape

(154014, 84)

In [11]:
y.shape

(154014,)

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

In [13]:
X_train.shape

(98568, 84)

In [14]:
X_test.shape

(30803, 84)

In [15]:
X_val.shape

(24643, 84)

### 1. Sampling the Features

In [16]:
from sklearn.ensemble import RandomForestRegressor

In [17]:
def feature_sampling_scores (max_features_list):
    feature_sampling_scores_list = []
    for max_feature in max_features_list:
        rfr = RandomForestRegressor(max_features=max_feature, random_state=1,n_estimators=10)
        rfr.fit(X_train, y_train)
        score = (max_feature, rfr.score(X_val,y_val))
        feature_sampling_scores_list.append(score)
    return feature_sampling_scores_list

In [18]:
%%time
feature_sampling_scores([0.5,'log2','sqrt',None])

CPU times: user 14.5 s, sys: 76.8 ms, total: 14.6 s
Wall time: 14.6 s


[(0.5, 0.9993452266555103),
 ('log2', 0.9285381816631885),
 ('sqrt', 0.961896030764962),
 (None, 0.9997430010136438)]

We can see from above that using None and 0.5 perform the best.

### 2. Number of Trees

In [37]:
rfr_n_trees = RandomForestRegressor(n_estimators=20, random_state=1, max_features=0.5)

In [39]:
%%time
rfr_n_trees.fit(X_train, y_train)

CPU times: user 9.53 s, sys: 30.9 ms, total: 9.56 s
Wall time: 9.58 s


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features=0.5, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=20,
                      n_jobs=None, oob_score=False, random_state=1, verbose=0,
                      warm_start=False)

In [40]:
rfr_n_trees.score(X_val,y_val)

0.9994767340764567

In [41]:
tree_predictions = np.vstack([estimator.predict(X_val) for estimator in rfr_n_trees.estimators_])

In [42]:
tree_predictions.shape

(20, 24643)

In [43]:
from sklearn.metrics import r2_score
r2_scores = [r2_score(y_val, np.mean(tree_predictions[:i + 1], axis = 0)) for i in range(0, len(rfr_n_trees.estimators_)) ]

In [44]:
from graph import trace_values, plot
x_vals = list(range(1, len(rfr_n_trees.estimators_) + 1))
trace = trace_values(x_vals, r2_scores)
plot([trace])

### 3. Number of leaves

In [21]:
min_samples = np.arange(2,51,1)
min_samples

array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50])

In [22]:
def num_leaves_wrapped_scores (min_samples):
    num_leaves_wrapped_scores_list = []
    for min_sample in min_samples:
        rfr = RandomForestRegressor(n_estimators=20, min_samples_leaf=min_sample, random_state=1)
        rfr.fit(X_train, y_train)
        score = rfr.score(X_val,y_val)
        num_leaves_wrapped_scores_list.append(score)
    return num_leaves_wrapped_scores_list

In [23]:
%%time
scores = num_leaves_wrapped_scores(min_samples)

CPU times: user 10min 38s, sys: 1.04 s, total: 10min 39s
Wall time: 10min 40s


In [24]:
scores[:5]

[0.9997908580206516,
 0.9997695509383336,
 0.9997312960993839,
 0.9997262976019109,
 0.9997006247990404]

In [29]:
wrapped_scores = np.column_stack((min_samples, scores))

In [45]:
trace_1 = trace_values(wrapped_scores[:, 0], wrapped_scores[:, 1])

In [46]:
plot([trace_1])

In [47]:
X_val.shape

(24643, 84)

In [48]:
combined_X = np.vstack((X_train, X_val))

In [49]:
combined_y = np.concatenate((y_train, y_val))

In [50]:
rfr3 = RandomForestRegressor(n_estimators=50, random_state=1, max_features=0.5)
rfr3.fit(combined_X, combined_y)
rfr3.score(X_test,y_test)

0.999640462886111